# 🧮 FP: Java Functional Interfaces

- What Action and Func family of delegates are doing
- Composability through methods like apply, before, after, andThen and compose

In [ ]:
Function<Integer, Integer> addOne = x -> x + 1;
Function<Integer, Integer> multiplyByTwo = x -> x * 2;

Function<Integer, Integer> combined = addOne.andThen(multiplyByTwo);

int result = combined.apply(3); // Result: (3 + 1) * 2 = 8

In [ ]:
Func<int, int> addOne = x => x + 1;
Func<int, int> multiplyByTwo = x => x * 2;

Func<int, int> combined = x => multiplyByTwo(addOne(x));

Func<int, int> combinedBetter = x =>
{
    x = addOne(x);
    return multiplyByTwo(x);
};

combinedBetter(3)

In [ ]:
public static Func<T, TResult> AndThen<T, TIntermediate, TResult>(this Func<T, TIntermediate> first,
        Func<TIntermediate, TResult> second) => x => second(first(x));

Func<int, int> addOne = x => x + 1;
Func<int, int> multiplyByTwo = x => x * 2;

var combined = addOne.AndThen(multiplyByTwo);

combined(3)

- https://github.com/MewsSystems/FuncSharp
- https://github.com/louthy/language-ext

# 🔣 FP: Algebraic Data Types (ADTs)

## 1- Product Types

A product type is a type that combines several other types
- Used to combine multiple values into a single type
- In C#, we can use the built-in Tuple classes (Value and Reference Tuples) to create product types

## 2- Sum Types / Tagged Unions

Sum Type (also known as Tagged Union or Discriminated Union) A sum type is a type that can be one of several possible types
- a type that can be one of several different types
- In C#, we can implement sum types using inheritance; we can use Pattern Matching to check the type

In [ ]:
abstract record Shape;

record Circle(float Radius) : Shape;
record Rectangle(float Width, float Height) : Shape;

static double Area(Shape shape) => shape switch
{
    Circle c => Math.PI * c.Radius * c.Radius,
    Rectangle r => r.Width * r.Height,
    _ => throw new ArgumentException("Unknown shape", nameof(shape))
};

- https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/discriminated-unions
- https://fsharpforfunandprofit.com/posts/type-size-and-design

In [ ]:
type Shape =
    | Rectangle of width : float * length : float
    | Circle of radius : float

let calculateArea shape =
    match shape with
    | Rectangle (width, length) -> width * length
    | Circle radius -> System.Math.PI * radius * radius

In [ ]:
type Direction =
    | North
    | East
    | South
    | West

type Movement =
    | Moving of Direction
    | NotMoving

type Optional<'a> =
    | Something of 'a
    | Nothing

type Either<'a,'b> =
    | Left of 'a
    | Right of 'b

## 3- Example

In [ ]:
// Pattern Matching and Deconstruction Weather Example from 03-b 👈
abstract record WeatherResponse;
record Success(string City, double Temperature, string Condition) : WeatherResponse;
record Sunny(string City, double Temperature) : Success(City, Temperature, "Sunny");
record NotFound(string City) : WeatherResponse;
record Error(string ErrorMessage) : WeatherResponse;

WeatherResponse GetWeatherResponse(string city) => city switch
{
    // Simulating different responses
    "Lahore" => new Sunny("Lahore", 49.0),
    "Faisalabad" => new Success("Faisalabad", 49.5, "Very Sunny"),
    not null => new NotFound(city),
    _ => new Error("Unknown error occurred")
};

# 🧩 FP: Putting it Altogether

## 1- Better Domain Layer

In [ ]:
using System.Diagnostics.Contracts;
using System.Threading.Tasks;

readonly struct Result<T>
{
    enum ResultState { Null, Failure, Success }
    readonly ResultState state;

    public T Value { get; }
    public Exception Exception { get; }

    public bool IsSuccess => this.state == ResultState.Success;
    public bool IsFailure => this.state == ResultState.Failure;
    public bool IsNull => this.state == ResultState.Null;

    public Result(T value)
    {
        this.Value = value;
        this.Exception = null!;
        this.state = ResultState.Success;
    }

    public Result(Exception exception)
    {
        this.Value = default!;
        this.Exception = exception;
        this.state = ResultState.Failure;
    }

    [Pure]
    public TR Match<TR>(Func<T, TR> onSuccess, Func<Exception, TR> onFailure, Func<TR>? onNull = null) =>
        IsSuccess ? onSuccess(Value) :
        IsFailure ? onFailure(Exception) :
        onNull is not null 
            ? onNull() 
            : throw new InvalidOperationException("Result is null, but no onNull function was provided.");

    public static implicit operator Result<T>(T? value) => value is not null ? new Result<T>(value) : new Result<T>();
    public static implicit operator Result<T>(Exception exception) => new(exception);
}

record Patient();
dynamic patientsRepository;
async Task<Result<Patient>> GetPatientAsync(string resourceId) 
{
    try
    {
        var response = await patientsRepository.GetAsync(resourceId);
        return response; // implicit conversion to Result<Patient> with a successful status and book as a Value
    }
    catch (Exception ex)
    {
        return ex; // implicit conversion to Result<Patient> with an exception so we have a failure status
    }
}

<img src=images/match.png>

- https://devblogs.microsoft.com/ise/next-level-clean-architecture-boilerplate/
- https://github.com/dorlugasigal/MiniClean.Template

## 2- Higher Order - Curried Functions - Helpers

- Redis Connection Retry
- Memoization
- Debouncing; we will see it today

## 3- Dynamic Programming

- Bottom Up
    - Tabulation
- Top Down
    - Memoization
- https://www.educative.io/blog/python-dynamic-programming-tutorial

# ❓Did you know

We can use throw as an expression

In [ ]:
// the conditional operator
string[] args = null;
string first = args.Length >= 1 
    ? args[0]
    : throw new ArgumentException("Please supply at least one argument");

// the null-coalescing operator
string name;
string Name
{
    get => name;
    set => name = value ??
        throw new ArgumentNullException(paramName: nameof(value), message: "Name cannot be null");
}

// expression bodied lambda
DateTime ToDateTime(IFormatProvider provider) =>
    throw new InvalidCastException("Conversion to a DateTime is not supported");

# 🔢Indexers

In [ ]:
class HundredCollection<T>
{
   T[] arr = new T[100];
   int nextIndex = 0;

   public T this[int i] => arr[i];

   public void Add(T value)
   {
      if (nextIndex >= arr.Length)
         throw new IndexOutOfRangeException($"The collection can hold only {arr.Length} elements.");
      arr[nextIndex++] = value;
   }
}

var hundredNumbers = new HundredCollection<int>();
hundredNumbers.Add(100);
hundredNumbers[1]       // any guesses what value it will be holding at index 1 ?

<img src=images/chess-board.jpg width=300>

In [ ]:
enum ChessPieceColor { White, Black }
enum ChessPieceType { Pawn, Rook, Knight, Bishop, Queen, King }
record ChessPiece(ChessPieceColor Color, ChessPieceType Type);

class ChessBoard
{
    ChessPiece[,] board = new ChessPiece[8, 8];

    public ChessPiece this[string square]
    {
        get
        {
            int row = 8 - int.Parse(square[1].ToString());
            int column = square[0] - 'a';

            if (row >= 0 && row < 8 && column >= 0 && column < 8)
                return board[row, column];
            else
                throw new ArgumentOutOfRangeException($"Invalid square: {square}");
        }
        set
        {
            int row = 8 - int.Parse(square[1].ToString());
            int column = square[0] - 'a';

            if (row >= 0 && row < 8 && column >= 0 && column < 8)
                board[row, column] = value;
            else
                throw new ArgumentOutOfRangeException($"Invalid square: {square}");
        }
    }
}

var board = new ChessBoard();

// Placing pieces on the board
board["e2"] = new ChessPiece(ChessPieceColor.White, ChessPieceType.Pawn);
board["d7"] = new ChessPiece(ChessPieceColor.Black, ChessPieceType.Pawn);

// Moving the piece on the board
board["e4"] = board["e2"];
board["e2"] = null;

__Resources__
- https://learn.microsoft.com/en-us/dotnet/csharp/programming-guide/indexers

# 🎁Namespace of the Day

## System.Collections.Frozen

__Immutable Collections__
- Collections that cannot be modified after they are created. Any modification operation returns a new collection with the modification applied, leaving the original collection unchanged
- Ideal for scenarios where collections are shared across multiple threads and need to be accessed in a thread-safe manner without the overhead of locks
- Immutable collections are designed to optimize the creation of new versions of the collection. This often involves sharing as much of the underlying data as possible with the original collection to minimize memory usage and copying overhead

__Frozen Collections__
- These collections are mutable during creation but become fixed (frozen) once their creation is complete. After freezing, the collection cannot be modified.
    - Once the Freeze method is called, the collection becomes immutable, and any further attempts to modify it will throw an exception.
- Best suited for collections that are constructed once and then used for lookups or iteration without modification. They provide optimized performance for lookup operations after the collection is frozen.
- Frozen collections are highly optimized for read operations, especially lookups. They can use more efficient data structures internally since the collection's size and contents are known to be fixed after freezing.


In [ ]:
using System.Collections.Immutable;
using System.Collections.Frozen;

var immutableList = ImmutableList.Create<int>();
var newList = immutableList.Add(1); // immutableList remains unchanged

var list = new List<int> { 1, 2, 3 };
var frozenSet = list.ToFrozenSet(); // frozenSet is now immutable

# 🧵System.Threading.Tasks

## 1- Threading 101 Recap

In [ ]:
// Java
Thread t = new Thread(new Runnable() { // Runnable is functional interface; we are declaring an anonymous inner class
    public void run() {
        // do something
    }
});

t.start();
// do something else
t.join();

In [ ]:
using System.Threading;

var thread = new Thread(() =>       // this is ThreadStart with inference; ThreadStart is delegate
{
    // do something
});

thread.Start();
// do something else
thread.Join();

## 2- System.Threading.Tasks

System.Threading.Tasks was introduced in .NET 4
- Task based modern APIs 
- Parallel Programming Features were introduced
- *Async Programming came later in .NET 4.5 (Windows Phone / Windows 8)* that continued to build upon Tasks API

In [ ]:
using static System.Console;
using static System.DateTime;
using System.Threading.Tasks;

var task1 = Task.Factory.StartNew(() => Task.Delay(3000));
var task2 = Task.Run(() => Task.Delay(5000));
// There's Thread.Sleep; its used with classic Threading API
// Task.Delay can be awaited but lets discuss async await later

WriteLine(Now);
Task.WaitAll(task1, task2); // we should complete these tasks in 5s
WriteLine(Now);

In [ ]:
using System.Threading;
using System.Threading.Tasks;

var task1 = Task.Run(() => 
{
    Task.Delay(2000).Wait();
    return 5;
});

var task2 = task1.ContinueWith(t1 => 
{
    //t1.IsCompletedSuccessfully
    //t1.IsFaulted
    //t1.Exception
    Console.WriteLine($"Task 2; from 1 we got {t1.Result}");
});

task2.Wait();

__Resources__
- https://learn.microsoft.com/en-us/dotnet/standard/parallel-programming/task-based-asynchronous-programming 👈

## 3- Debouncing

- Higher Order Function
- Task

In [ ]:
using static System.Console;
using System.Threading;
using System.Threading.Tasks;

static Action<T> Debounce<T>(Action<T> func, int milliseconds = 300)
{
    var last = 0;
    return arg =>
    {
        var current = Interlocked.Increment(ref last);
        Task.Delay(milliseconds).ContinueWith(task =>
        {
            if (current == last) func(arg);
            task.Dispose();
        });
    };
}

Action<string> search = input => WriteLine($"Searching {input}");
Action<string> debouncedSearch = Debounce<string>(s => search(s), 500);

debouncedSearch("Go");
debouncedSearch("Goog");
debouncedSearch("Google");

// giving the debounced function time to complete
Thread.Sleep(1000);

- Currying

In [ ]:
static Func<Action<T>, Action<T>> CurriedDebounce<T>(int milliseconds = 300) =>
    func => Debounce(func, milliseconds);

Action<string> search = input => WriteLine($"Searching {input}");
var debouncer = CurriedDebounce<string>(300); //debouncer is a function
var debouncedSearch = debouncer(search);

debouncedSearch("Go");
debouncedSearch("Goog");
debouncedSearch("Google");

// giving the debounced function time to complete
Thread.Sleep(1000);